# Notebook Title

## Setup Python and R environment
you can ignore this section

In [20]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [22]:
%%R

# My commonly used R imports

require('tidyverse')

## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [23]:
geocode_df=pd.read_csv("geocoded_col_10k.csv")

## 👉 Grab Census Data

1. loading the Census API key

In [24]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [25]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [28]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
11 B07011

In [ ]:
%%R -o nyc_census_data
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
nyc_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        population="B01003_001",
                        med_inc="B19013_001",
                        means_transit_car="B08301_002E",
                        means_transit_bus="B08301_011E",
                        school_kids="B14001_002E",
                        kids="B09001_001E"
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=F)

nyc_census_data

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

In [38]:
%%R -o nyc_census_data
nyc_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        population="B01003_001",
                        med_inc="B19013_001",
                        means_transit_car="B08301_002",
                        means_transit_bus="B08301_011",
                        school_kids="B14001_002",
                        kids="B09001_001"
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=F)

nyc_census_data

# A tibble: 13,962 × 5
   GEOID       NAME                                   variable    estimate   moe
   <chr>       <chr>                                  <chr>          <dbl> <dbl>
 1 36005000100 Census Tract 1, Bronx County, New York population      6661   702
 2 36005000100 Census Tract 1, Bronx County, New York means_tran…        0    18
 3 36005000100 Census Tract 1, Bronx County, New York means_tran…        0    18
 4 36005000100 Census Tract 1, Bronx County, New York kids              98    77
 5 36005000100 Census Tract 1, Bronx County, New York school_kids       27    25
 6 36005000100 Census Tract 1, Bronx County, New York med_inc           NA    NA
 7 36005000200 Census Tract 2, Bronx County, New York population      4453   563
 8 36005000200 Census Tract 2, Bronx County, New York means_tran…      779   380
 9 36005000200 Census Tract 2, Bronx County, New York means_tran…      235   150
10 36005000200 Census Tract 2, Bronx County, New York kids            1008   234
# ℹ 1

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [39]:
%%R 

# pivot wide 
nyc_census_data <- nyc_census_data %>% 
    pivot_wider(names_from = "variable", values_from = "estimate")

In [40]:
nyc_census_data

,GEOID,NAME,variable,estimate,moe
1,36005000100,"Census Tract 1, Bronx County, New York",population,6661.0,702.0
2,36005000100,"Census Tract 1, Bronx County, New York",means_transit_car,0.0,18.0
3,36005000100,"Census Tract 1, Bronx County, New York",means_transit_bus,0.0,18.0
4,36005000100,"Census Tract 1, Bronx County, New York",kids,98.0,77.0
5,36005000100,"Census Tract 1, Bronx County, New York",school_kids,27.0,25.0
...,...,...,...,...,...
13958,36085990100,"Census Tract 9901, Richmond County, New York",means_transit_car,0.0,13.0
13959,36085990100,"Census Tract 9901, Richmond County, New York",means_transit_bus,0.0,13.0
13960,36085990100,"Census Tract 9901, Richmond County, New York",kids,0.0,13.0
13961,36085990100,"Census Tract 9901, Richmond County, New York",school_kids,0.0,13.0


In [41]:
%%R

geo_code <- read.csv("geocoded_col_10k.csv")

# Merge the dataframes
merged_data <- merge(nyc_census_data, geo_code, by = "GEOID", all.x = TRUE)

In [42]:
%%R

# write to CSV so we can view it in python
merged_data %>% 
    write_csv("merged_data.csv")

In [43]:
df=pd.read_csv("merged_data.csv")
df.dtypes

GEOID                              int64
NAME                              object
moe                              float64
population                       float64
means_transit_car                float64
means_transit_bus                float64
kids                             float64
school_kids                      float64
med_inc                          float64
CRASH.DATE                        object
CRASH.TIME                        object
BOROUGH                           object
ZIP.CODE                         float64
LATITUDE                         float64
LONGITUDE                        float64
LOCATION                          object
ON.STREET.NAME                    object
CROSS.STREET.NAME                 object
OFF.STREET.NAME                   object
NUMBER.OF.PERSONS.INJURED        float64
NUMBER.OF.PERSONS.KILLED         float64
NUMBER.OF.PEDESTRIANS.INJURED    float64
NUMBER.OF.PEDESTRIANS.KILLED     float64
NUMBER.OF.CYCLIST.INJURED        float64
NUMBER.OF.CYCLIS